# Libraries

In [1]:
from collections import Counter

#import matplotlib.pyplot as plt
import numpy as np
#import pandas as pd

import random
import time
#import seaborn as sns

#%matplotlib inline
#pd.options.display.mpl_style = 'default'
#sns.set(style="whitegrid")

## Loading the data

Data taken from https://www.kaggle.com/c/digit-recognizer

Note: making a load function and classifier from scratch is quite innefficient in terms of RAM and compute time. To accomodate this I cut down the original data using linux commands from 42,000 rows to 1000:

```
$ wc -l train.csv
>>> 42001 train.csv

$ head -n 1000 train.csv > train_first1000.csv
```

In [2]:
def load_data(data_dir, train_file):
             
             #test_file):
    """
    Loads the training file and testing file into numpy arrays from given
    directory.
    
    Separates out the header, and the data from target.
    """
    # Read in train file.
    train_data = open(data_dir + train_file).read()
    
    # Split rows based on newline character.
    train_data = train_data.split("\n")
    
    # The first row is the header.
    header = train_data[0].split(',')
    
    # From row 1 to the end is the data.
    train_data = train_data[1:-1]
    
    # Inside each row, split values based on commas.
    train_data = [i.split(",") for i in train_data]
    
    # y is the first column, X is the rest of the columns.
    # Convert values from string to integers and store them in numpy arrays.
    y_train = np.array([int(i[0]) for i in train_data])
    X_train = np.array([[int(i[j]) for j in range(1,len(i))] for i in train_data])

    # Same for test file.
    #test_data = open(data_dir + test_file).read()
    #test_data = test_data.split("\n")[1:-1]
    #test_data = [i.split(",") for i in test_data]
    #X_test = np.array([[int(i[j]) for j in range(0,len(i))] for i in test_data])

    return header, X_train, y_train

In [3]:
# Directory and file details.
data_dir = "../data/"
train_file = "train_first1000.csv"
#test_file = "test_first1000.csv"

In [4]:
# Load in using function.
header, X_train, y_train = load_data(data_dir, train_file)

In [5]:
# Inspecting the header.
print header[0:5], header[-5:-1]

['label', 'pixel0', 'pixel1', 'pixel2', 'pixel3'] ['pixel779', 'pixel780', 'pixel781', 'pixel782']


In [64]:
X_train[0]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0, 170, 255, 114,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0, 114, 255, 25

In [65]:
X_train.shape

(700, 784)

The reduced data files contains 999 instances of digits. The first column "label" is the actual class of the hand-drawn digit, from zero through nine. The proceeding columns make-up the image. Each image is 28 pixels in height and 28 pixels in width, for a total of 784 pixels in total. Each pixel has a single pixel-value associated with it, indicating the lightness or darkness of that pixel, with higher numbers meaning darker. This pixel-value is an integer between 0 and 255, inclusive.

## Splitting the data

In [10]:
# Take a list of the indices and randomize the order.
indices = np.random.permutation(X_train.shape[0])

In [11]:
# Set the proportion of data saved for testing purposes.
test_proportion = 0.3

# Get the amount that corresponds to proportion
test_amount = int(X_train.shape[0] * test_proportion)
train_amount = X_train.shape[0] - test_amount

In [12]:
# Take the first 70% of indices as training, last 30% as testing.
training_idx, test_idx = indices[:train_amount], indices[test_amount:]

In [13]:
# Split the data and target into training and testing.
X_train, X_test, y_train, y_test = X_train[training_idx,:], X_train[test_idx,:], y_train[training_idx], y_train[test_idx]

## Defining KNN model

In [57]:
class knn():
    """ A kNN classifier with L2 distance. """

    def __init__(self):
        pass

    def train(self, X, y):
        """
        Trains the classifier. For k-nearest neighbors this is just 
        memorizing the training data.
        
        Inputs:
        
        - X: A numpy array of shape (num_train, D) containing the training data
          consisting of num_train samples each of dimension D.
        - y: A numpy array of shape (N,) containing the training labels, where
             y[i] is the label for X[i].
        """
        # Remember the training data.
        self.X_train = X
        self.y_train = y

    def predict(self, X, k=1):
        """
        Predict labels for test data using this classifier.
        
        Inputs:
        - X: A numpy array of shape (num_test, D) containing test data consisting
             of num_test samples each of dimension D.
        - k: The number of nearest neighbors that vote for the predicted labels.
        - num_loops: Determines which implementation to use to compute distances
          between training points and testing points.
          
        Returns:
        - y: A numpy array of shape (num_test,) containing predicted labels for the
          test data, where y[i] is the predicted label for the test point X[i].  
        """

        # Compute the distance between each point
        dists = self.compute_distances(X)
        
        # Init array to hold label predictions.
        num_test = dists.shape[0]
        y_pred = np.zeros(num_test)

        for i in range(num_test):
            
            # Init array to hold k closest labels.
            k_closest_y = []
            
            # Sort dists and take corresponding labels.
            labels = self.y_train[np.argsort(dists[i,:])].flatten()
            
            # Find k nearest labels.
            k_closest_y = labels[:k]

            # Take most common label. In case of a tie take the smaller value.
            c = Counter(k_closest_y)
            y_pred[i] = c.most_common(1)[0][0]

        return y_pred
    
    def compute_distances(self, X):
        """
        Compute the distance between each test point in X and each training point
        in self.X_train using a nested loop over both the training data and the 
        test data.
        
        Inputs:
        - X: A numpy array of shape (num_test, D) containing test data.
        
        Returns:
        - dists: A numpy array of shape (num_test, num_train) where dists[i, j]
          is the Euclidean distance between the ith test point and the jth training
          point.
        """
        # Number of test and training points.
        num_test = X.shape[0]
        num_train = self.X_train.shape[0]
        
        # Init empty array to hold distances.
        dists = np.zeros((num_test, num_train)) 

        # Dot product between points.
        dot_pro = np.dot(X, self.X_train.T)
        
        # Test and train matrices squared.
        sum_square_test = np.square(X).sum(axis = 1)
        sum_square_train = np.square(self.X_train).sum(axis = 1)
        
        # Vectorized Euclidean distance. (http://stackoverflow.com/posts/37903795/revisions)
        dists = np.sqrt(-2 * dot_pro + sum_square_train + np.matrix(sum_square_test).T)
        
        return dists

## Training the model

In [58]:
# Set number of nearest neighbours to take.
k = 5

# Initialize KNN classiier.
classifier = knn()

# Train the model.
classifier.train(X_train, y_train)

## Evaluating the model

In [59]:
predictions = classifier.predict(X_test, k)

In [63]:
accuracy = (predictions == y_test).sum() / float(len(predictions)) * 100
print "{0:.2f}%".format(accuracy)

88.43%


C:\Users\Chris\AppData\Roaming\Python\Python27\site-packages\ipykernel\__main__.py:1: RuntimeWarning: invalid value encountered in equal
  if __name__ == '__main__':


In [55]:
type(int(predictions[0]))

int

In [47]:
type(y_test[0])

numpy.int32